In [2]:
# example of training a stable gan for generating a handwritten digit
import tensorflow as tf
from os import makedirs
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.initializers import RandomNormal
from matplotlib import pyplot
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
# weight initialization
 init = RandomNormal(stddev=0.02)
# define model
 model = Sequential()
# downsample to 14x14
 model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, input_shape=in_shape))
 model.add(LeakyReLU(alpha=0.2))
# downsample to 7x7
 model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
 
 model.add(LeakyReLU(alpha=0.2))
# classifier
 model.add(Flatten())
 model.add(Dense(1, activation='sigmoid'))
# compile model
 opt = Adam(lr=0.0002, beta_1=0.5)
 model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
 return model
# define the standalone generator model
def define_generator(latent_dim):
# weight initialization
 init = RandomNormal(stddev=0.02)
# define model
 model = Sequential()
# foundation for 7x7 image
 n_nodes = 128 * 7 * 7
 model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
 model.add(LeakyReLU(alpha=0.2))
 model.add(Reshape((7, 7, 128)))
# upsample to 14x14
 model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',kernel_initializer=init))
 
 model.add(LeakyReLU(alpha=0.2))
# upsample to 28x28
 model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',kernel_initializer=init))
 
 model.add(LeakyReLU(alpha=0.2))
# output 28x28x1
 model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
 return model
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
# make weights in the discriminator not trainable
 discriminator.trainable = False
# connect them
 model = Sequential()
# add generator
 model.add(generator)
# add the discriminator
 model.add(discriminator)
# compile model
 opt = Adam(lr=0.0002, beta_1=0.5)
 model.compile(loss='binary_crossentropy', optimizer=opt)
 return model
# load mnist images
def load_real_samples():
# load dataset
 (trainX, trainy), (_, _) = load_data()
# expand to 3d, e.g. add channels
 X = expand_dims(trainX, axis=-1)
# select all of the examples for a given class
 selected_ix = trainy == 8
 X = X[selected_ix]
# convert from ints to floats
 X = X.astype('float32')
# scale from [0,255] to [-1,1]
 X = (X - 127.5) / 127.5
 return X
# select real samples
def generate_real_samples(dataset, n_samples):
# choose random instances
 ix = randint(0, dataset.shape[0], n_samples)
# select images
 X = dataset[ix]
# generate class labels
 y = ones((n_samples, 1))
 return X, y
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
# generate points in the latent spac
 x_input = randn(latent_dim * n_samples)
# reshape into a batch of inputs for the network
 x_input = x_input.reshape(n_samples, latent_dim)
 return x_input
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
# generate points in latent space
 x_input = generate_latent_points(latent_dim, n_samples)
# predict outputs
 X = generator.predict(x_input)
# create class labels
 y = zeros((n_samples, 1))
 return X, y
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
# prepare fake examples
 X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
# scale from [-1,1] to [0,1]
 X = (X + 1) / 2.0
# plot images
 for i in range(10 * 10):
# define subplot
  pyplot.subplot(10, 10, 1 + i)
# turn off axis
  pyplot.axis('off')
# plot raw pixel data
  pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
# save plot to file
 pyplot.savefig('results_baseline/generated_plot_%03d.png' % (step+1))
 pyplot.close()
# save the generator model
 g_model.save('results_baseline/model_%03d.h5' % (step+1))
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
# plot loss
 pyplot.subplot(2, 1, 1)
 pyplot.plot(d1_hist, label='d-real')
 pyplot.plot(d2_hist, label='d-fake')
 pyplot.plot(g_hist, label='gen')
 pyplot.legend()
# plot discriminator accuracy
 pyplot.subplot(2, 1, 2)
 pyplot.plot(a1_hist, label='acc-real')
 pyplot.plot(a2_hist, label='acc-fake')
 pyplot.legend()
# save plot to file
 pyplot.savefig('results_baseline/plot_line_plot_loss.png')
 pyplot.close()
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
# calculate the number of batches per epoch
 bat_per_epo = int(dataset.shape[0] / n_batch)
# calculate the total iterations based on batch and epoch
 n_steps = bat_per_epo * n_epochs
# calculate the number of samples in half a batch
 half_batch = int(n_batch / 2)
# prepare lists for storing stats each iteration
 d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
# manually enumerate epochs
 for i in range(n_steps):
# get randomly selected ✬real✬ samples
  X_real, y_real = generate_real_samples(dataset, half_batch)
# update discriminator model weights
  d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
# generate ✬fake✬ examples
  X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
# update discriminator model weights
  d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
# prepare points in latent space as input for the generator
  X_gan = generate_latent_points(latent_dim, n_batch)
# create inverted labels for the fake samples
  y_gan = ones((n_batch, 1))
# update the generator via the discriminator✬s error
  g_loss = gan_model.train_on_batch(X_gan, y_gan)
# summarize loss on this batch
  print('>%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
          (i+1, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
# record history
  d1_hist.append(d_loss1)
  d2_hist.append(d_loss2)
  g_hist.append(g_loss)
  a1_hist.append(d_acc1)
  a2_hist.append(d_acc2)
# evaluate the model performance every ✬epoch✬
  if (i+1) % bat_per_epo == 0:
     summarize_performance(i, g_model, latent_dim)
 plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)
# make folder for results
makedirs('results_baseline', exist_ok=True)
# size of the latent space
latent_dim = 50
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
print(dataset.shape)
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

(5851, 28, 28, 1)
>1, d1=0.681, d2=0.694 g=0.692, a1=95, a2=0
>2, d1=0.641, d2=0.695 g=0.692, a1=100, a2=0
>3, d1=0.605, d2=0.696 g=0.690, a1=100, a2=0
>4, d1=0.571, d2=0.700 g=0.687, a1=100, a2=0
>5, d1=0.534, d2=0.707 g=0.680, a1=100, a2=0
>6, d1=0.493, d2=0.718 g=0.669, a1=100, a2=0
>7, d1=0.451, d2=0.738 g=0.652, a1=100, a2=0
>8, d1=0.411, d2=0.767 g=0.628, a1=100, a2=0
>9, d1=0.375, d2=0.809 g=0.600, a1=100, a2=0
>10, d1=0.349, d2=0.854 g=0.580, a1=100, a2=0
>11, d1=0.331, d2=0.876 g=0.577, a1=100, a2=0
>12, d1=0.335, d2=0.863 g=0.594, a1=100, a2=0
>13, d1=0.341, d2=0.822 g=0.631, a1=100, a2=0
>14, d1=0.351, d2=0.769 g=0.683, a1=100, a2=0
>15, d1=0.356, d2=0.705 g=0.747, a1=100, a2=1
>16, d1=0.337, d2=0.636 g=0.826, a1=100, a2=100
>17, d1=0.363, d2=0.573 g=0.913, a1=100, a2=100
>18, d1=0.357, d2=0.508 g=1.012, a1=100, a2=100
>19, d1=0.336, d2=0.447 g=1.120, a1=100, a2=100
>20, d1=0.346, d2=0.393 g=1.234, a1=100, a2=100
>21, d1=0.319, d2=0.342 g=1.357, a1=100, a2=100
>22, d1=0.319,

>175, d1=0.644, d2=0.658 g=0.878, a1=62, a2=73
>176, d1=0.702, d2=0.655 g=0.865, a1=50, a2=75
>177, d1=0.652, d2=0.645 g=0.856, a1=70, a2=79
>178, d1=0.719, d2=0.649 g=0.854, a1=51, a2=76
>179, d1=0.668, d2=0.653 g=0.865, a1=56, a2=76
>180, d1=0.707, d2=0.653 g=0.846, a1=53, a2=73
>181, d1=0.671, d2=0.673 g=0.842, a1=57, a2=65
>182, d1=0.682, d2=0.655 g=0.834, a1=53, a2=67
>183, d1=0.648, d2=0.668 g=0.852, a1=73, a2=67
>184, d1=0.699, d2=0.653 g=0.847, a1=51, a2=71
>185, d1=0.753, d2=0.661 g=0.837, a1=46, a2=68
>186, d1=0.662, d2=0.652 g=0.838, a1=64, a2=70
>187, d1=0.701, d2=0.676 g=0.836, a1=57, a2=57
>188, d1=0.638, d2=0.677 g=0.834, a1=65, a2=60
>189, d1=0.660, d2=0.643 g=0.848, a1=54, a2=71
>190, d1=0.688, d2=0.678 g=0.860, a1=54, a2=60
>191, d1=0.717, d2=0.682 g=0.830, a1=46, a2=50
>192, d1=0.694, d2=0.663 g=0.807, a1=54, a2=65
>193, d1=0.672, d2=0.693 g=0.818, a1=65, a2=51
>194, d1=0.709, d2=0.720 g=0.800, a1=56, a2=35
>195, d1=0.718, d2=0.720 g=0.812, a1=53, a2=50
>196, d1=0.69

>350, d1=0.749, d2=0.716 g=0.718, a1=28, a2=42
>351, d1=0.760, d2=0.706 g=0.710, a1=21, a2=46
>352, d1=0.749, d2=0.728 g=0.714, a1=28, a2=32
>353, d1=0.733, d2=0.717 g=0.717, a1=29, a2=35
>354, d1=0.740, d2=0.710 g=0.717, a1=25, a2=42
>355, d1=0.743, d2=0.717 g=0.715, a1=28, a2=42
>356, d1=0.749, d2=0.709 g=0.719, a1=21, a2=42
>357, d1=0.740, d2=0.717 g=0.697, a1=26, a2=39
>358, d1=0.748, d2=0.724 g=0.731, a1=18, a2=31
>359, d1=0.736, d2=0.714 g=0.713, a1=32, a2=40
>360, d1=0.736, d2=0.739 g=0.712, a1=25, a2=31
>361, d1=0.736, d2=0.728 g=0.714, a1=25, a2=34
>362, d1=0.755, d2=0.713 g=0.712, a1=15, a2=37
>363, d1=0.755, d2=0.722 g=0.715, a1=15, a2=34
>364, d1=0.742, d2=0.720 g=0.709, a1=28, a2=32
>365, d1=0.729, d2=0.723 g=0.721, a1=29, a2=35
>366, d1=0.746, d2=0.719 g=0.715, a1=26, a2=39
>367, d1=0.752, d2=0.714 g=0.725, a1=20, a2=40
>368, d1=0.745, d2=0.724 g=0.711, a1=25, a2=32
>369, d1=0.748, d2=0.705 g=0.700, a1=20, a2=50
>370, d1=0.736, d2=0.720 g=0.717, a1=26, a2=43
>371, d1=0.74